In [0]:
import re

volume_path = "/Volumes/workspace/default/my_data_volume"

files = dbutils.fs.ls(volume_path)
pattern = re.compile(r"\d{4}-[a-zA-Z]+-points\.csv")

candidate_files = [ f for f in files if pattern.match(f.name)]



In [0]:
#Bronze ingestion
from pyspark.sql.functions import current_timestamp, lit
from pyspark.sql.functions import col
from pyspark.sql.utils import AnalysisException

catalog = "workspace"
schema = "bronze"
table = "tennis_points_raw"

#Create Unity Catalog schema
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {catalog}.{schema}")

try:
    bronze_df = spark.table(f"{catalog}.{schema}.{table}")
    table_exists = True
except AnalysisException:
    table_exists = False


for f in candidate_files:
    file_name = f.name
    file_path = f.path
    archive_path = f"{volume_path}/archive/{file_name}"

    files_already_ingested = False

    if table_exists:
        files_already_ingested = (
            bronze_df.filter(col("source_file") == file_name).limit(1).count() > 0
        )

    if files_already_ingested:
        dbutils.fs.mv(file_path, archive_path)
        print(f"File {file_name} already ingested. Archiving then Skipping.")
        continue

    raw_df = (
        spark.read
        .option("header", "true")
        .option("inferSchema", "true")
        .csv(file_path)
    )

    new_bronze_df = (
        raw_df
        .withColumn("source_file", lit(file_name))
        .withColumn("ingest_timestamp", current_timestamp())
    )

    new_bronze_df = new_bronze_df.withColumn("ElapsedTime", col("ElapsedTime").cast("string"))

    (
        new_bronze_df.write
        .format("delta")
        .mode("append")
        .saveAsTable(f"{catalog}.{schema}.{table}")
    )

    dbutils.fs.mv(file_path, archive_path)
    print(f"File {file_name} ingested and archived.")



In [0]:
# %sql
# SELECT source_file, COUNT(*) AS row_count
# FROM workspace.bronze.tennis_points_raw
# GROUP BY source_file
# ORDER BY source_file;


In [0]:
%sql
-- CREATE OR REPLACE TEMP VIEW bronze_dedup AS
-- SELECT *
-- FROM (
--   SELECT *,
--          ROW_NUMBER() OVER (
--            PARTITION BY source_file, match_id, PointNumber
--            ORDER BY ingest_timestamp ASC
--          ) AS rn
--   FROM workspace.bronze.tennis_points_raw
-- )
-- WHERE rn = 1;

-- CREATE OR REPLACE TABLE workspace.bronze.tennis_points_raw
-- USING DELTA
-- AS
-- SELECT
--   * EXCEPT (rn)
-- FROM bronze_dedup;




In [0]:
# %sql
# SELECT
#   source_file,
#   COUNT(DISTINCT ingest_timestamp) AS ingest_runs,
#   COUNT(*) AS total_rows
# FROM workspace.bronze.tennis_points_raw
# GROUP BY source_file;
